In [2]:
import nbimporter
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
import numpy as np
import gc

In [3]:
def generate_features(dataset):
    '''
    Генерация фичей
    Вход: - сырой датасет
    Выход: - обработанный датасет с данными
    '''
    ### Добавляем новый столбец для отображения временной регулярности, с которой появлялись данные в исходном наборе данных
    dataset['seq_time'] = np.arange(dataset.shape[0], dtype=np.float32)
    
    # функция генерирует профильные признаки на основе идентификатора пользователя или песни (item).
    # Признаки включают статистические характеристики, такие как среднее и стандартное отклонение для непрерывных переменных 
    # continuous_vars и отношение частот для категориальных переменных category_vars.
    def create_profile(item, continuous_vars, category_vars):
        dataframes_profile = pd.DataFrame({item: dataset[item], item + '_count': dataset.groupby(item)[item].transform('count')})
        for featur in continuous_vars:
            dataframes_profile[item + '_' + featur + '_mean'] = dataset.groupby(item)[featur].transform('mean')
            dataframes_profile.loc[:, item + '_' + featur + '_std'] = dataset.groupby(item)[featur].transform('std')
        for featur in category_vars:
            dataframes_profile[item + '_' + featur + '_ratio'] = dataset.groupby([item, featur])[featur].transform('count')/dataframes_profile[item+'_count']
        for col in dataframes_profile.columns:
            if col != item:
                dataframes_profile[col] = dataframes_profile[col].astype('float32')
        return dataframes_profile
    
    
    # инициализация пользователя
    item = 'msno'
    continuous_vars = ['song_length']
    category_vars = ['genre_ids', 'language', 'registered_via', 'source_screen_name', 'source_system_tab', 'source_type']
    dataset_profile_msno = create_profile(item, continuous_vars, category_vars)

    # инициализация песен
    item = 'song_id'
    continuous_vars = ['bd', 'expiration_date', 'register_period', 'registration_init_time']
    category_vars = ['city', 'gender', 'registered_via', 'source_screen_name', 'source_system_tab', 'source_type']
    dataframe_profile_song_id = create_profile(item, continuous_vars, category_vars)
    
    ### get latent features
    # Добавление столбца times_song_id с подсчётом количества раз, когда каждая пара пользователь-песня (msno-song_id) встречается в наборе данных
    dataframes_latents = dataset[['msno', 'song_id', 'artist_name']]
    dataframes_latents['times_song_id'] = dataframes_latents.groupby(['msno', 'song_id'])['msno'].transform('count')
    dataframes_latents['times_artist_name'] = dataframes_latents.groupby(['msno', 'artist_name'])['msno'].transform('count')

    # Создание словаря all_msno,all_song_id, all_artist_name который отображает каждую уникальную идентификационную 
    # метку пользователя (msno) на числовой индекс
    all_msno = sorted(dataframes_latents['msno'].unique())
    all_msno = dict(zip(all_msno, range(len(all_msno))))
    
    all_song_id = sorted(dataframes_latents['song_id'].unique())
    all_song_id = dict(zip(all_song_id, range(len(all_song_id))))
    
    all_artist_name = sorted(dataframes_latents['artist_name'].unique())
    all_artist_name = dict(zip(all_artist_name, range(len(all_artist_name))))
    
    dataframes_latents['ind_msno'] = dataframes_latents['msno'].map(all_msno)
    dataframes_latents['ind_song_id'] = dataframes_latents['song_id'].map(all_song_id)
    dataframes_latents['ind_artist_name'] = dataframes_latents['artist_name'].map(all_artist_name)

    df_copy = dataframes_latents[['msno', 'song_id', 'ind_msno', 'ind_song_id', 'times_song_id']].copy()

    # Создаем разряженную матрицу mat_artist_name(data,(row,col)). Хороший пример показан вот тут clck.ru/3ARSwz
    mat_song_id = csr_matrix((df_copy['times_song_id'], (df_copy['ind_msno'], df_copy['ind_song_id'])), 
                              shape=(len(all_msno), len(all_song_id)))
    # Снижаем размерность матрицы с помощью сокращенного сингулярного разложения
    svd_song_id = TruncatedSVD(n_components=20, random_state=0)
    svd_song_id.fit(mat_song_id)
    # вычисляем сумму долей объясненной дисперсии компонентами SVD. Каждый компонент имеет свою долю объясненной дисперсии (информации), и 
    # суммирование этих долей дает общее количество информации, содержащейся во всех выбранных компонентах.
    svd_song_id.explained_variance_ratio_.sum() 

    # добавляем в датафрейм категориальные имена художников в одном столбце и соответствующие латентные признаки (компоненты SVD) в остальных столбцах.
    latents_msnos = pd.DataFrame(svd_song_id.transform(mat_song_id))
    latents_msnos.columns = ['svd_msno1_'+str(i) for i in range(latents_msnos.shape[1])]
    latents_msnos = pd.concat((pd.DataFrame({'msno': all_msno.keys()}), latents_msnos), axis=1)
    latents_msnos['msno'] = latents_msnos['msno'].astype('category')
    latents_msnos.loc[:, latents_msnos.columns[1:]] = latents_msnos.loc[:, latents_msnos.columns[1:]].astype('float32')

    # тоже самое что и latents_msnos
    latent_song_id = pd.DataFrame(svd_song_id.components_.T)
    latent_song_id.columns = ['svd_song_id_'+str(i) for i in range(latent_song_id.shape[1])]
    latent_song_id = pd.concat((pd.DataFrame({'song_id': all_song_id.keys()}), latent_song_id), axis=1)
    latent_song_id['song_id'] = latent_song_id['song_id'].astype('category')
    latent_song_id.loc[:, latent_song_id.columns[1:]] = latent_song_id.loc[:, latent_song_id.columns[1:]].astype('float32')

    # т.к. таблицы ОГРОМНЫЕ, нужно вызывать сборщик мусора, чтобы хоть как-то всё запускалось
    del mat_song_id, df_copy
    gc.collect()

    # Такие же шаги что и в mat_song_id
    mat_artist_name = csr_matrix((dataframes_latents['times_artist_name'], (dataframes_latents['ind_msno'], dataframes_latents['ind_artist_name'])), 
                                  shape=(len(all_msno), len(all_artist_name)))
    # Снижаем размерность матрицы с помощью сокращенного сингулярного разложения
    svd_artist_name = TruncatedSVD(n_components=10, random_state=0)
    svd_artist_name.fit(mat_artist_name)
    # вычисляем сумму долей объясненной дисперсии компонентами SVD. Каждый компонент имеет свою долю объясненной дисперсии (информации), и 
    # суммирование этих долей дает общее количество информации, содержащейся во всех выбранных компонентах.
    svd_artist_name.explained_variance_ratio_.sum()

    # добавляем в датафрейм категориальные имена художников в одном столбце и соответствующие латентные признаки (компоненты SVD) в остальных столбцах.
    latents_msnos2 = pd.DataFrame(svd_artist_name.transform(mat_artist_name))
    latents_msnos2.columns = ['svd_msno2_'+str(i) for i in range(latents_msnos2.shape[1])]
    latents_msnos2 = pd.concat((pd.DataFrame({'msno': all_msno.keys()}), latents_msnos2), axis=1)
    latents_msnos2['msno'] = latents_msnos2['msno'].astype('category')
    latents_msnos2.loc[:, latents_msnos2.columns[1:]] = latents_msnos2.loc[:, latents_msnos2.columns[1:]].astype('float32')

    # тоже самое что и latents_msnos2
    latent_artist_names = pd.DataFrame(svd_artist_name.components_.T)
    latent_artist_names.columns = ['svd_artist_name_'+str(i) for i in range(latent_artist_names.shape[1])]
    latent_artist_names = pd.concat((pd.DataFrame({'artist_name': all_artist_name.keys()}), latent_artist_names), axis=1)
    latent_artist_names['artist_name'] = latent_artist_names['artist_name'].astype('category')
    latent_artist_names.loc[:, latent_artist_names.columns[1:]] = latent_artist_names.loc[:, latent_artist_names.columns[1:]].astype('float32')


    ### Мёржим весь датафрейм в 1
    dataset = pd.concat((dataset, dataset_profile_msno.loc[:,dataset_profile_msno.columns!='msno']), axis=1)
    dataset = pd.concat((dataset, dataframe_profile_song_id.loc[:,dataframe_profile_song_id.columns!='song_id']), axis=1)
    dataset = dataset.merge(latents_msnos, how='left', on='msno')
    dataset = dataset.merge(latent_song_id, how='left', on='song_id')
    dataset = dataset.merge(latents_msnos2, how='left', on='msno')
    dataset = dataset.merge(latent_artist_names, how='left', on='artist_name')
    
    # вновь удаляем мусор
    del svd_song_id, latents_msnos, latent_song_id, svd_artist_name, latents_msnos2, latent_artist_names
    gc.collect()
    
    return dataset